# Chatbot RAG - Économie Française
Ce projet porte la mise en place d’un chatbot intelligent capable d’interroger un rapport PDF en langage naturel.
L’architecture repose sur une approche RAG (Retrieval-Augmented Generation), qui combine la recherche d’informations pertinentes dans un document et la génération de réponses contextuelles grâce à un modèle de langage.

Technologies utilisées :

- Ollama (Mistral) : génération d’embeddings et production de réponses en français

- FAISS : moteur de recherche vectorielle pour retrouver les passages les plus pertinents du document

- Streamlit : interface web interactive et conviviale permettant d’interagir avec le chatbot

## Création des fichiers docs.index et docs.json pour l’indexation des documents

In [1]:
# Usage: exécution après avoir lancé "ollama serve" et téléchargé le modèle mistral.

from pypdf import PdfReader
import ollama
import numpy as np
import faiss
import json
import os
from tqdm import tqdm  # pas obligatoir mais pratique pour voir l'avancement

# Configuration 
pdf_path = "/home/sacko/Documents/ChatbotEcoFranceRAG/Fichiers/ECO_FRANCE.pdf"   # chemin vers le fichier PDF
model = "mistral"
chunk_size = 800       # taille en caractères par chunk 
chunk_overlap = 200    # recouvrement entre chunks
index_file = "docs.index"
docs_file = "docs.json"

## Extraction automatique du contenu d’un rapport PDF pour l’analyse IA

In [2]:
# Extraction du contenu textuel d’un PDF
reader = PdfReader(pdf_path)
pages = [p.extract_text() for p in reader.pages]
pages = [p for p in pages if p]  # Exclusion des pages vides lors de l’extraction
full_text = "\n\n".join(pages)
print(f"Texte extrait : {len(full_text)} caractères, {len(pages)} pages")

Texte extrait : 24313 caractères, 13 pages


## Prétraitement et découpage des textes afin d’alimenter le moteur RAG

In [3]:
# Découpage en chunks
def chunk_text(text, chunk_size=800, overlap=200):
    chunks = []
    start = 0
    L = len(text)
    while start < L:
        end = min(start + chunk_size, L)
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

chunks = chunk_text(full_text, chunk_size=chunk_size, overlap=chunk_overlap)
print(f"{len(chunks)} chunks créés (chunk_size={chunk_size}, overlap={chunk_overlap})")

41 chunks créés (chunk_size=800, overlap=200)


##  Production des vecteurs d’embedding via Ollama

In [4]:
# On s'assure que le serveur Ollama est en cours d’exécution et que le modèle mistral a bien été téléchargé (ollama pull mistral).
embeddings = []
for chunk in tqdm(chunks, desc="Embeddings"):
    resp = ollama.embed(model=model, input=chunk)
    
    # Le nom de la clé varie selon la version du SDK : embedding pour un vecteur unique ou embeddings pour une liste. Notre code gère les deux.    if "embeddings" in resp:
    if "embeddings" in resp:
        emb = resp["embeddings"][0]
        
    elif "embedding" in resp:
        emb = resp["embedding"]
        
    else:
        raise RuntimeError("Format d'embedding inattendu : %s" % resp)
    embeddings.append(emb)
            
embeddings = np.array(embeddings, dtype="float32")
print("Embeddings shape:", embeddings.shape)

Embeddings: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [04:25<00:00,  6.48s/it]

Embeddings shape: (41, 4096)


## Génération et stockage de l’index FAISS

In [5]:
# Indexation FAISS et enregistrement du fichier d’index
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)  # index simple L2 
index.add(embeddings)
faiss.write_index(index, index_file)
print("Index sauvegardé:", index_file)

# 5) Sauvegarde des chunks (id -> texte)
with open(docs_file, "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)
print("Chunks sauvegardés:", docs_file)

Index sauvegardé: docs.index
Chunks sauvegardés: docs.json


# Notebook interactif

In [7]:
# Charger l'index et les chunks
index_file = "/home/sacko/Documents/ChatbotEcoFranceRAG/Scripts/docs.index"
docs_file = "/home/sacko/Documents/ChatbotEcoFranceRAG//Scripts/docs.json"
model = "mistral"
index = faiss.read_index(index_file)
with open(docs_file, "r", encoding="utf-8") as f:
    docs = json.load(f)

In [8]:
def embed_query(text, model="mistral"):
    resp = ollama.embed(model=model, input=text)
    # gestion de deux formats possibles
    if "embeddings" in resp:
        return np.array(resp["embeddings"][0], dtype="float32")
    elif "embedding" in resp:
        return np.array(resp["embedding"], dtype="float32")
    else:
        raise RuntimeError("Format d'embedding inattendu : %s" % resp)

In [9]:
def retrieve_context(query, k):
    query_vector = embed_query(query)
    query_vector = np.expand_dims(query_vector, axis=0)
    distances, indices = index.search(query_vector, k)
    
    results = []
    for dist, idx in zip(distances[0], indices[0]):
        results.append({
            "id": int(idx),
            "distance": float(dist),
            "text": chunks[idx]  # chaque chunk est une string
        })
    return results

In [12]:
# Pour construire le prompt
query = "Quels facteurs influencent l'économie française ?"

retrieved = retrieve_context(query, k = 4)

retrieved_texts = [r["text"] for r in retrieved]

def build_prompt(query, retrieved_chunks):
    context = "\n\n".join(retrieved_chunks)
    return f"Contexte :\n{context}\n\nQuestion : {query}\nRéponse :"

prompt = build_prompt(query, retrieved_texts)

resp = ollama.chat(model="mistral", messages=[{"role":"user","content":prompt}])
print(resp["message"]["content"])

 Les facteurs qui influencent l'économie française sont multiples et peuvent être regroupés en plusieurs catégories.

Dans le contexte de votre texte, on peut identifier :

1. La politique économique allemande d'augmentation de la compétitivité-coût des entreprises en réduisant les cotisations sociales et en augmentant l'imposition sur les biens importés. Cela a pu avoir un impact négatif sur l'économie française, en renforçant les gains de parts de marché allemands au détriment de la France, ce qui s'est traduit par un ralentissement de l'activité dans l'Hexagone.

2. Les effets économiques des crises financières, comme celle des années 2008 et 2009, avec des déficits importants et une dette publique considérablement accrue en Europe.

3. Les innovations technologiques et la diffusion du modèle de production fordiste dans l'industrie ont également eu un impact important sur l'économie française. Cela a permis d'améliorer la productivité du travail, ce qui a conduit à une croissance éc